# Household Location Choice Model (HLCM) for Single and Multi Family Housing  


Arezoo Besharati, Paul Waddell, UrbanSim, July 2018 

This notebook demonstrates the use of the LargeMultinomialLogit model template to construct, estimate, and evaluate a Household Location Choice Model for the San Francisco Bay Area.

In the process of developing the model, we also demonstrate some data checking and transformations to improve the model.

The model structure and specification are informed and limited by the available data, which is based on the data used by the Metropolitan Transportation Commission for their operational model.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Initialize-Networks" data-toc-modified-id="Initialize-Networks-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Initialize Networks</a></span></li><li><span><a href="#Explore-the-data" data-toc-modified-id="Explore-the-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Explore the data</a></span></li><li><span><a href="#Data-Cleaning" data-toc-modified-id="Data-Cleaning-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Data Cleaning</a></span></li><li><span><a href="#Perform-desired-variable-creations-and-transformations¶" data-toc-modified-id="Perform-desired-variable-creations-and-transformations¶-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Perform desired variable creations and transformations¶</a></span></li><li><span><a href="#Check-Distributions" data-toc-modified-id="Check-Distributions-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Check Distributions</a></span></li><li><span><a href="#Check-for-Multicollinearity" data-toc-modified-id="Check-for-Multicollinearity-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Check for Multicollinearity</a></span></li></ul></li><li><span><a href="#Model-Estimation" data-toc-modified-id="Model-Estimation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model Estimation</a></span><ul class="toc-item"><li><span><a href="#HLCM-for-Multi-Family" data-toc-modified-id="HLCM-for-Multi-Family-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>HLCM for Multi-Family</a></span></li><li><span><a href="#HLCM-for-Single-Family" data-toc-modified-id="HLCM-for-Single-Family-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>HLCM for Single-Family</a></span></li></ul></li></ul></div>

## Preliminaries

In [1]:
import os; os.chdir('../../')
import numpy as np, pandas as pd 
import matplotlib.pyplot as plt
import warnings;
warnings.simplefilter('ignore')
%load_ext autoreload
%autoreload 2

In [2]:
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import orca
import seaborn as sns
%matplotlib notebook

In [3]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

### Load data

In [4]:
# Load any script-based Orca registrations
from scripts import datasources
from scripts import models

In [ ]:
orca.list_tables()
#orca.list_steps()

In [5]:
orca.broadcast('nodeswalk', 'rentals', cast_index=True, onto_on='node_id_walk')
orca.broadcast('nodeswalk', 'parcels', cast_index=True, onto_on='node_id_walk')
orca.broadcast('nodessmall', 'rentals', cast_index=True, onto_on='node_id_walk')
orca.broadcast('nodessmall', 'parcels', cast_index=True, onto_on='node_id_walk')
orca.list_broadcasts()

[('parcels', 'buildings'),
 ('buildings', 'units'),
 ('units', 'households'),
 ('households', 'persons'),
 ('nodeswalk', 'rentals'),
 ('nodeswalk', 'parcels'),
 ('nodessmall', 'rentals'),
 ('nodessmall', 'parcels')]

In [ ]:
#mm.list_steps()

In [ ]:
#mm.initialize()

### Initialize Networks

Note that we have separately created a set of network accessibility variables and stored them. They are registered a orca objects.

In [ ]:
#%%capture
#orca.run(["initialize_network_walk"])

In [ ]:
#%%capture
#orca.run(["initialize_network_small"])

### Explore the data

In [6]:
parcel = orca.get_table('parcels').to_frame()
parcel.columns

Index(['development_type_id', 'land_value', 'acres', 'county_id', 'zone_id',
       'proportion_undevelopable', 'tax_exempt_status', 'apn',
       'parcel_id_local', 'geom_id', 'imputation_flag', 'x', 'y', 'shape_area',
       'block_id', 'node_id', 'node_id_small', 'node_id_walk'],
      dtype='object')

In [7]:
bld = orca.get_table('buildings').to_frame()
#bld.columns

In [13]:
hh = orca.get_table('households').to_frame()
#hh.columns.tolist()

In [17]:
hh['hh_random'] = np.random.uniform(0,1,len(hh))

In [18]:
hh.head()

,serialno,persons,building_type,cars,income,race_of_head,hispanic_head,age_of_head,workers,state,...,tract,block_group,children,tenure,recent_mover,block_group_id,single_family,unit_id,building_id,hh_random
household_id,,,,,,,,,,,,,,,,,,,,,
0,2010000487191,1,6.00,1.00,85000.00,1,no,47,1.00,6,...,500901,1,0.00,2,0,60855009011,False,1711366,409174,0.88
1,2013000554587,1,9.00,1.00,27000.00,6,no,52,1.00,6,...,500901,1,0.00,2,0,60855009011,False,1711818,1579443,0.55
2,2011001140920,2,2.00,1.00,6000.00,6,no,60,1.00,6,...,500901,1,0.00,2,0,60855009011,True,1711727,1365639,0.44
3,2012001376432,1,3.00,1.00,28000.00,8,yes,51,1.00,6,...,500901,1,0.00,2,0,60855009011,True,1711548,647211,0.91
4,2013000186929,1,6.00,1.00,10000.00,1,no,64,0.00,6,...,500901,1,0.00,2,0,60855009011,False,1711121,309586,0.23


In [19]:
hh.describe()

,serialno,persons,building_type,cars,income,race_of_head,age_of_head,workers,children,tenure,recent_mover,unit_id,building_id,hh_random
count,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00,2677468.00
mean,2011048463205.79,2.64,3.59,1.88,113128.36,2.84,52.97,1.30,0.56,1.43,0.04,1288427.13,858095.84,0.50
std,1416905562.31,1.57,2.34,1.14,107952.60,2.66,15.69,1.00,0.97,0.49,0.19,849779.84,561165.80,0.29
min,2009000000022.00,1.00,1.00,0.00,-11600.00,1.00,15.00,0.00,0.00,1.00,0.00,-1.00,-1.00,0.00
25%,2010000424551.00,1.00,2.00,1.00,40320.00,1.00,41.00,1.00,0.00,1.00,0.00,540970.75,359954.75,0.25
50%,2011000848265.00,2.00,2.00,2.00,84000.00,1.00,53.00,1.00,0.00,1.00,0.00,1272990.50,837858.50,0.50
75%,2012001211486.00,4.00,5.00,2.00,150100.00,6.00,63.00,2.00,1.00,2.00,0.00,2024749.25,1341263.25,0.75
max,2013001492566.00,20.00,10.00,6.00,1397000.00,9.00,94.00,13.00,9.00,2.00,1.00,2781489.00,1843272.00,1.00


In [27]:
hh.dtypes

serialno            int64
persons             int64
building_type     float64
cars              float64
income            float64
race_of_head        int64
hispanic_head      object
age_of_head         int64
workers           float64
state              object
county             object
tract              object
block_group        object
children          float64
tenure              int64
recent_mover        int64
block_group_id     object
single_family        bool
unit_id             int64
building_id         int64
hh_random         float64
dtype: object

In [38]:
hh_f = hh[(hh['building_type'] > 2) & (hh['hh_random'] < .2) & (hh['recent_mover'] == 1) \
         & (hh['income'] > 0) & (hh['income'] < 500000)]
len(hh_f)

15308

In [ ]:
#bld.groupby('building_type_id')['residential_units'].sum()

In [ ]:
#bld.describe().transpose()

In [ ]:
#bld.quantile(.99)

In [ ]:
#bld[['residential_units','residential_sqft', 'non_residential_sqft']].sum()

In [ ]:
#bld.building_type_id.value_counts()

In [ ]:
# hh_sf=hh[hh['building_type']==2]
# hh_mf=hh[hh['building_type']!=2]

### Data Cleaning

In [ ]:
# nodeswalk = orca.get_table('nodeswalk').to_frame()
# nodessmall = orca.get_table('nodessmall').to_frame()
#print(nodeswalk.columns.tolist())
#print(nodessmall.columns.tolist())

In [ ]:
#nodessmall.describe().transpose()

In [ ]:
# nodessmall_upper = nodessmall.quantile(.99)
#nodessmall_upper

In [ ]:
# nodessmall_clipped = nodessmall.clip_upper(nodessmall_upper, axis=1) 
# nodessmall_clipped.describe().transpose()
# orca.add_table('nodessmall', nodessmall_clipped);

In [ ]:
# nodeswalk.describe().transpose()
# nodeswalk_upper = nodeswalk.quantile(.99)
#nodeswalk_upper

In [ ]:
# nodeswalk_clipped = nodeswalk.clip_upper(nodeswalk_upper, axis=1) 
# nodeswalk_clipped.describe().transpose()
# orca.add_table('nodeswalk', nodeswalk_clipped);

### Perform desired variable creations and transformations¶

In [22]:
# scale income and create race dummies
hh.income_k = hh.income/1000
hh.white = (hh.race_of_head == 1).astype(int)
hh.black = (hh.race_of_head == 2).astype(int)
hh.asian = (hh.race_of_head == 6).astype(int)
hh.hisp = (hh.hispanic_head == 'yes').astype(int)
hh.single = (hh.persons == 1).astype(int)
hh.elderly = (hh.age_of_head > 65).astype(int)

  
# building_type dummies
bld.single_family = (bld.building_type_id == 1).astype(int)
bld.multi_family = (bld.building_type_id == 3).astype(int)
bld.mixed_use = (bld.building_type_id > 3).astype(int)

# add the columns

orca.add_column('households', 'income_k', hh.income_k)
orca.add_column('households', 'white', hh.white)
orca.add_column('households', 'black', hh.black)
orca.add_column('households', 'asian', hh.asian)
orca.add_column('households', 'hispanic', hh.hisp)
orca.add_column('households', 'elderly', hh.elderly)
orca.add_column('households', 'single', hh.single)
orca.add_column('households', 'hh_random', hh.hh_random)

orca.add_column('buildings', 'single_family', bld.single_family)
orca.add_column('buildings', 'multi_family', bld.multi_family)
orca.add_column('buildings', 'mixed_use', bld.mixed_use);

In [ ]:
# nodeswalk['prop_children_500_walk'] = (nodeswalk['children_500_walk'] > 0).astype(int) / nodeswalk['hh_500_walk']
# nodeswalk['prop_singles_500_walk'] = nodeswalk['singles_500_walk'] / nodeswalk['hh_500_walk']
# nodeswalk['prop_elderly_500_walk'] = nodeswalk['elderly_hh_500_walk'] / nodeswalk['hh_500_walk']
# nodeswalk['prop_black_500_walk'] = nodeswalk['pop_black_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_white_500_walk'] = nodeswalk['pop_white_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_asian_500_walk'] = nodeswalk['pop_asian_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_hisp_500_walk'] = nodeswalk['pop_hisp_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_rich_500_walk'] = nodeswalk['rich_500_walk'] / nodeswalk['pop_500_walk']
# nodeswalk['prop_poor_500_walk'] = nodeswalk['poor_500_walk'] / nodeswalk['pop_500_walk']

# nodeswalk['prop_children_1500_walk'] = (nodeswalk['children_1500_walk'] > 0).astype(int) / nodeswalk['hh_1500_walk']
# nodeswalk['prop_singles_1500_walk'] = nodeswalk['singles_1500_walk'] / nodeswalk['hh_1500_walk']
# nodeswalk['prop_elderly_1500_walk'] = nodeswalk['elderly_hh_1500_walk'] / nodeswalk['hh_1500_walk']
# nodeswalk['prop_black_1500_walk'] = nodeswalk['pop_black_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_white_1500_walk'] = nodeswalk['pop_white_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_asian_1500_walk'] = nodeswalk['pop_asian_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_hisp_1500_walk'] = nodeswalk['pop_hisp_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_rich_1500_walk'] = nodeswalk['rich_1500_walk'] / nodeswalk['pop_1500_walk']
# nodeswalk['prop_poor_1500_walk'] = nodeswalk['poor_1500_walk'] / nodeswalk['pop_1500_walk']


# orca.add_column('nodeswalk', 'prop_children_500_walk', nodeswalk['prop_children_500_walk'])
# orca.add_column('nodeswalk', 'prop_singles_500_walk', nodeswalk['prop_singles_500_walk'])
# orca.add_column('nodeswalk', 'prop_elderly_500_walk', nodeswalk['prop_elderly_500_walk'])
# orca.add_column('nodeswalk', 'prop_black_500_walk', nodeswalk['prop_black_500_walk'])
# orca.add_column('nodeswalk', 'prop_white_500_walk', nodeswalk['prop_white_500_walk'])
# orca.add_column('nodeswalk', 'prop_asian_500_walk', nodeswalk['prop_asian_500_walk'])
# orca.add_column('nodeswalk', 'prop_hisp_500_walk', nodeswalk['prop_hisp_500_walk'])
# orca.add_column('nodeswalk', 'prop_rich_500_walk', nodeswalk['prop_rich_500_walk'])
# orca.add_column('nodeswalk', 'prop_poor_500_walk', nodeswalk['prop_poor_500_walk'])

# orca.add_column('nodeswalk', 'prop_children_1500_walk', nodeswalk['prop_children_1500_walk'])
# orca.add_column('nodeswalk', 'prop_singles_1500_walk', nodeswalk['prop_singles_1500_walk'])
# orca.add_column('nodeswalk', 'prop_elderly_1500_walk', nodeswalk['prop_elderly_1500_walk'])
# orca.add_column('nodeswalk', 'prop_black_1500_walk', nodeswalk['prop_black_1500_walk'])
# orca.add_column('nodeswalk', 'prop_white_1500_walk', nodeswalk['prop_white_1500_walk'])
# orca.add_column('nodeswalk', 'prop_asian_1500_walk', nodeswalk['prop_asian_1500_walk'])
# orca.add_column('nodeswalk', 'prop_hisp_1500_walk', nodeswalk['prop_hisp_1500_walk'])
# orca.add_column('nodeswalk', 'prop_rich_1500_walk', nodeswalk['prop_rich_1500_walk'])
# orca.add_column('nodeswalk', 'prop_poor_1500_walk', nodeswalk['prop_poor_1500_walk'])

### Check Distributions

In [ ]:
# hh_income_upper = hh_singlefamily.income.quantile(0.99)
# len(hh[hh['income']>hh_income_upper])

In [ ]:
# %matplotlib notebook
# hh_sf.income.plot(kind='hist',bins=100)
# print(hh_sf.income.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
# %matplotlib notebook
# hh_mf.income.plot(kind='hist',bins=100)
# print(hh_mf.income.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
# %matplotlib notebook
# ax = sns.boxplot(x = hh_sf.income,palette="Set2", width=0.5)

In [ ]:
# %matplotlib notebook
# ax = sns.boxplot(x = hh_mf.income,palette="Set2", width=0.5)

In [ ]:
df = orca.merge_tables(target = 'buildings', tables = ['buildings','parcels','nodeswalk','nodessmall'])
#df.columns.tolist()
df_sf= df[df['residential_units']==1]
df_mf= df[df['residential_units']>1]

In [ ]:
# %matplotlib notebook
# df_sf.avg_income_1500_walk.plot(kind='hist',bins=100)
# plt.show()
# print(df_sf.avg_income_1500_walk.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
# %matplotlib notebook
# df_mf.avg_income_1500_walk.plot(kind='hist',bins=100)
# plt.show()
# print(df_mf.avg_income_1500_walk.quantile([0.10,0.5,0.9,0.99,1]))

In [ ]:
#%matplotlib notebook
#(df_mf.prop_black_500_walk).plot(kind='hist',bins=100)
#(df_mf.prop_asian_500_walk).plot(kind='hist',bins=100)
#(df_mf.prop_white_500_walk).plot(kind='hist',bins=100)
#(df_mf.prop_hisp_500_walk).plot(kind='hist',bins=100)


### Check for Multicollinearity

In [ ]:
# %matplotlib inline
# plt.subplots(figsize=(20, 20))


# # Create correlation matrix
# corr_matrix = df_sf.corr().abs()

# # Select upper triangle of correlation matrix
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# high_corr = [column for column in upper.columns if any(upper[column] > 0.98)]
# X= df_sf[high_corr]
# sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")

In [ ]:
# %matplotlib inline
# plt.subplots(figsize=(20, 20))

# corr_matrix = df_mf.corr()

# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# high_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
# X= df_mf[high_corr]
# plt.subplots(figsize=(20, 20))
# sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")

In [ ]:
%matplotlib inline

selected_features = ['residential_units','res_price_per_sqft', 'pop_1500_walk', 'jobs_1500_walk',
                     'prop_singles_1500_walk','prop_elderly_1500_walk','prop_children_1500_walk',
                     'avg_income_1500_walk','prop_black_1500_walk', 'sqft_per_unit','prop_poor_1500_walk',
                     'jobs_25000','prop_white_1500_walk','prop_black_1500_walk','prop_asian_1500_walk',
                     'prop_hisp_1500_walk']

X = df_mf[selected_features]

plt.subplots(figsize=(12, 12))
sns.heatmap(X.corr(), annot=True, cmap="RdYlGn")
plt.show()

## Model Estimation

### HLCM for Multi-Family 

In [23]:
%%time
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['buildings','parcels','nodeswalk','nodessmall']
m.choice_column = 'building_id'
m.alt_sample_size = 50

#Filters on choosers
m.chooser_filters = ['building_type > 2 &\
                      hh_random < .2 &\
                      recent_mover == 1 &\
                      0 <income < 500000']

#Filters on alternatives
m.alt_filters = ['residential_units > 1',
                 '0 < avg_income_500_walk < 500000',
                 'pop_1500_walk > 0',
                 'sqft_per_unit > 0']


m.model_expression = ' np.log(residential_units) + \
np.log1p(res_price_per_sqft) + \
np.log1p(sqft_per_unit) + \
np.log1p(income):np.log1p(sqft_per_unit) + \
np.log1p(jobs_1500_walk) + \
np.log1p(jobs_25000) + \
np.log(income):np.log(avg_income_1500_walk) + \
np.log1p(pop_1500_walk) + \
white:prop_white_500_walk + \
black:prop_black_500_walk + \
asian:prop_asian_500_walk + \
hispanic:prop_hisp_500_walk\
- 1'

m.name = 'hlcm'
m.tags = ['multi_family','test']
m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:            455
Model:         Multinomial Logit   Df Residuals:                443
Method:       Maximum Likelihood   Df Model:                     12
Date:                 2018-07-20   Pseudo R-squ.:             0.334
Time:                      16:44   Pseudo R-bar-squ.:         0.328
AIC:                   2,393.378   Log-Likelihood:       -1,184.689
BIC:                   2,442.822   LL-Null:              -1,779.970
                                                 coef   std err         z     P>|z|   Conf. Int.
------------------------------------------------------------------------------------------------
np.log(residential_units)                      1.0703     0.044    24.248     0.000             
np.log1p(res_price_per_sqft)                  -0.2152     0.051    -4.206     0.000             
np.log1p(sqft_per_unit)                       -0.4491     0.190    -

### HLCM for Single-Family 

In [ ]:
%%time
m = LargeMultinomialLogitStep()
m.choosers = ['households']
m.alternatives = ['buildings','parcels','nodeswalk','nodessmall']
m.choice_column = 'building_id'
m.alt_sample_size = 50

#Filters on choosers
m.chooser_filters = ['building_type == 2 & household_id <3000000 & recent_mover == 1 & 0 <income < 1000000']

#Filters on alternatives
m.alt_filters = ['residential_units == 1',
                 '0 < avg_income_500_walk < 500000',
                 'sqft_per_unit > 0']

# np.log(residential_units) +
# np.log(income):np.log(avg_income_500_walk) + \
# np.log1p(income):np.log1p(rich_1500_walk) + \
# np.log1p(income):np.log1p(poor_1500_walk) + \
# np.log1p(persons):np.log1p(sqft_per_unit) + \

m.model_expression = ' \
np.log1p(res_price_per_sqft) + \
np.log1p(income):np.log1p(res_price_per_sqft) + \
np.log1p(sqft_per_unit) + \
np.log1p(income):np.log1p(sqft_per_unit) + \
np.log1p(acres) + \
np.log1p(income):np.log1p(acres) + \
np.log1p(pop_1500_walk) + \
np.log1p(jobs_1500_walk) + \
np.log1p(jobs_25000) + \
single:np.log1p(singles_1500_walk) + \
elderly:np.log1p(elderly_hh_1500_walk) + \
children:np.log1p(children_1500_walk) + \
white:np.log1p(pop_white_1500_walk) + \
black:np.log1p(pop_black_1500_walk) + \
asian:np.log1p(pop_asian_1500_walk) + \
hispanic:np.log1p(pop_hisp_1500_walk) \
- 1'

m.name = 'hlcm'
m.tags = ['single_family', 'test']
m.fit()